# FP8 Quantization-Aware Training (QAT) with Unsloth

This notebook demonstrates how to perform **FP8 Quantization-Aware Training (QAT)** using the Unsloth library for efficient LLM fine-tuning.

### What is FP8 QAT?
- **FP8 (8-bit Floating Point)**: A reduced-precision format that uses only 8 bits instead of 16/32 bits, enabling faster computation and lower memory usage
- **Quantization-Aware Training**: During training, the model simulates quantization effects (fake quantization) so it learns to maintain accuracy despite reduced precision
- **Benefit**: After training, the model can be quantized to FP8 with minimal accuracy loss because it was trained to handle quantization noise

### LoRA Fine-Tuning with FP8 QAT
This notebook uses **LoRA (Low-Rank Adaptation)** combined with FP8 QAT. Only the LoRA adapter weights are trained (~1% of parameters), making training more memory-efficient while the model learns to be robust to FP8 quantization.

In [1]:
import os
os.environ["TORCH_CUDA_ARCH_LIST"] = "12.1"
os.environ["TRITON_PTXAS_PATH"] = "/usr/local/cuda/bin/ptxas"
os.environ["PATH"] = "/usr/local/cuda/bin:" + os.environ.get("PATH", "")
os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda/lib64:" + os.environ.get("LD_LIBRARY_PATH", "")

## Configure CUDA Environment

Set up environment variables for CUDA compilation and execution. This ensures the correct GPU architecture (compute capability 12.1) is targeted and that CUDA tools are accessible in the system PATH.

In [2]:
%pip install torch==2.9.0 torchvision==0.24.0 torchaudio==2.9.0 --index-url https://download.pytorch.org/whl/cu130
%pip install --upgrade unsloth unsloth_zoo
%pip install torchao==0.14.0 fbgemm-gpu-genai==1.4.2
%pip install transformers==4.55.4
%pip install --no-deps trl==0.22.2
%pip install -v -U --no-build-isolation -U git+https://github.com/facebookresearch/xformers.git@main#egg=xformers

Looking in indexes: https://download.pytorch.org/whl/cu130
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached torchao-0.14.0-py3-none-any.whl.metadata (19 kB)
ERROR: Could not find a version that satisfies the requirement fbgemm-gpu-genai==1.4.2 (from versions: none)
ERROR: No matching distribution found for fbgemm-gpu-genai==1.4.2
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Using pip 25.3 from /root/src/github.com/elizabetht/language-modeling-from-scratch/.venv/lib/python3.12/site-packages/pip (python 3.12)
  Cloning https://github.com/facebookresearch/xformers.git (to revision main) to /tmp/pip-install-yn_iyxhr/xformers_4c58cdf03b72474fba99052074d70c83
  Running command git version
  git version 2.43.0
  Running command git clone --filter

## Install Dependencies

Install PyTorch with CUDA support and all required libraries including Unsloth, transformers, TRL (for training), and torchao/fbgemm for FP8 quantization support.

In [3]:
import torch
from unsloth import FastLanguageModel

MODEL_ID="meta-llama/Llama-3.1-8B-Instruct"
max_seq_length = 2048
dtype = torch.bfloat16
lora_rank = 32

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_ID,
    dtype=dtype,
    max_seq_length=max_seq_length,
    load_in_4bit=False,
    fast_inference=False,
    max_lora_rank = lora_rank
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/root/src/github.com/elizabetht/language-modeling-from-scratch/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GB10 which is of cuda capability 12.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (8.0) - (12.0)
    
  warnings.warn(


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.3: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    NVIDIA GB10. Num GPUs = 1. Max memory: 119.635 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu130. CUDA: 12.1. CUDA Toolkit: 13.0. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Load the Pre-trained Model

Load the Llama-3.1-8B-Instruct model using Unsloth's `FastLanguageModel`. Key parameters:
- `dtype=torch.bfloat16`: Use bfloat16 precision for model weights
- `load_in_4bit=False`: Load model in full precision (not 4-bit quantized)
- `max_lora_rank=32`: Maximum LoRA rank for adapter layers

In [4]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
    

## Display Model Architecture

Print the model architecture to see the layer structure before applying LoRA adapters.

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules= [
        "q_proj", "k_proj", "v_proj", "o_proj", 
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha = lora_rank * 2,
    use_gradient_checkpointing = "unsloth",
    random_state = 3047,
    qat_scheme = "fp8-int4"
)

Unsloth 2026.1.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth: Applying QAT to mitigate quantization degradation


## Apply LoRA with FP8 QAT

Apply Low-Rank Adaptation (LoRA) adapters to the model with FP8 Quantization-Aware Training enabled. Key parameters:
- `r=32`: LoRA rank (determines adapter capacity)
- `target_modules`: Which layers get LoRA adapters (attention and MLP)
- `qat_scheme="fp8-int4"`: Enable FP8 fake quantization during training
- `lora_alpha=64`: LoRA scaling factor (typically 2x the rank)

In [6]:

for module in model.modules():
    if "FakeQuantized" in module.__class__.__name__:
        print("QAT is applied!")
        break

QAT is applied!


## Configure Chat Template

Set up the Llama-3 chat template for the tokenizer. This formats conversations with proper system/user/assistant tokens required for instruction following.

In [7]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama3"
)

### Load Dataset

Load the FineTome-100k dataset from Hugging Face, which contains multi-turn conversations for instruction-following tasks.

In [8]:
from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

### Load Training Dataset

Load the FineTome-100k dataset from Hugging Face for fine-tuning. This dataset contains high-quality instruction-following conversations.

In [9]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

### Standardize Data Format

Convert the dataset to a standardized conversation format compatible with Unsloth's training pipeline.

In [10]:
dataset[100]

{'conversations': [{'content': 'What is the modulus operator in programming and how can I use it to calculate the modulus of two given numbers?',
   'role': 'user'},
  {'content': 'In programming, the modulus operator is represented by the \'%\' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:\n\n```python\n# Calculate the modulus\nModulus = a % b\n\nprint("Modulus of the given numbers is: ", Modulus)\n```\n\nIn this code snippet, the variables \'a\' and \'b\' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator \'%\', we calculate the remainder when \'a\' is divided by \'b\'. The result is then stored in the variable \'Modulus\'. Finally, the modulus value is printed using the \'print\' statement.\n\nFor example, if \'a\' is 10 and \'b\' is 4, the modulus calculation would be 10 % 4, which equals 2. Theref

### Inspect Dataset Sample

View a sample from the dataset to verify the data structure and content quality.

In [11]:
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched = True)

### Preview Formatted Text

View the formatted text output for a sample to verify the chat template was applied correctly with proper conversation formatting.

In [12]:
dataset[100]['text']

'<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nWhat is the modulus operator in programming and how can I use it to calculate the modulus of two given numbers?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nIn programming, the modulus operator is represented by the \'%\' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:\n\n```python\n# Calculate the modulus\nModulus = a % b\n\nprint("Modulus of the given numbers is: ", Modulus)\n```\n\nIn this code snippet, the variables \'a\' and \'b\' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator \'%\', we calculate the remainder when \'a\' is divided by \'b\'. The result is then stored in the variable \'Modulus\'. Finally, the modulus value is printed using the \'print\' statement.\n\nFor example, if \'a\' is 10 and \'b\' is 4, the modulus 

### Test Text Formatting

Apply the formatting function to see how the conversation text is structured with the chat template.

In [13]:
model = model.to("cuda")

### Fast Language Model Training Preparation

Prepare the model for fast training with Unsloth optimizations and LoRA configuration.

## Start Training with FP8 QAT

Begin the training process using Unsloth's SFTTrainer with the following configuration:
- **Learning Rate**: 2e-5 (conservative for LoRA fine-tuning)
- **Batch Size**: 1 with gradient accumulation of 4 (effective batch size: 4)
- **Optimizer**: AdamW 8-bit for memory efficiency
- **Training Steps**: 30 (short demo run)
- **FP8 QAT**: Fake quantization enabled during training

In [14]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    # dataset_text_field = "text",
    # max_seq_length = max_seq_length,
    # packing = False,
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-5,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

[trl.trainer.sft_trainer|WARNING]You are using a per_device_train_batch_size of 1 with padding-free training. Using a batch size of 1 anihilate the benefits of padding-free training. Please consider increasing the batch size to at least 2.


🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


Use Unsloth's train_on_completions method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [15]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",
    response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Verify masking the instruction part is done! Let's print the 100th row again.

Print the masked out example

In [16]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                                 In programming, the modulus operator is represented by the \'%\' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:\n\n```python\n# Calculate the modulus\nModulus = a % b\n\nprint("Modulus of the given numbers is: ", Modulus)\n```\n\nIn this code snippet, the variables \'a\' and \'b\' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator \'%\', we calculate the remainder when \'a\' is divided by \'b\'. The result is then stored in the variable \'Modulus\'. Finally, the modulus value is printed using the \'print\' statement.\n\nFor example, if \'a\' is 10 and \'b\' is 4, the modulus calculation would be 10 % 4, which equals 2. Therefore, the output of the above code would be:\n\n```\nModulus of the given numbers is: 2\n```\n\nThis means that the modulus of 10 and 4 is 2.<|eot_i

In [17]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nWhat is the modulus operator in programming and how can I use it to calculate the modulus of two given numbers?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nIn programming, the modulus operator is represented by the \'%\' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:\n\n```python\n# Calculate the modulus\nModulus = a % b\n\nprint("Modulus of the given numbers is: ", Modulus)\n```\n\nIn this code snippet, the variables \'a\' and \'b\' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator \'%\', we calculate the remainder when \'a\' is divided by \'b\'. The result is then stored in the variable \'Modulus\'. Finally, the modulus value is printed using the \'print\' statement.\n\nFor example, if \'a\' is 10 and \'b\' i

Let's train the model! 

In [18]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 83,886,080 of 8,114,147,328 (1.03% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.817200
2,0.719900
3,0.819500
4,0.724000
5,1.101000
6,0.964000
7,0.771800
8,0.860800
9,0.912800
10,0.575900


## Convert Fake Quantization to Real Quantization

This is the critical step where we convert the fake quantization (used during training) into real FP8 quantized weights. 

During training, the model used simulated quantization - the weights were quantized and immediately dequantized for the forward pass, allowing gradients to flow properly. Now we convert this simulation into actual FP8 compressed weights.

The `step="convert"` parameter tells TorchAO to:
- Remove the fake quantization wrappers
- Apply real FP8 quantization to the weights
- Prepare the model for deployment with compressed weights

Now that training is complete, let's convert the FakeQuantizedLinear layers back to standard nn.Linear layers. This removes the fake quantization overhead and prepares the model for its final conversion step or for merging LoRA adapters.

In [19]:
from torchao.quantization import quantize_
from torchao.quantization.qat import QATConfig

quantize_(model, QATConfig(step = "convert"))

## Test Inference with Quantized Model

Test the fine-tuned and quantized model with a sample prompt to verify it generates coherent responses after FP8 conversion. This ensures the quantization process didn't degrade the model's performance significantly.

In [20]:
messages = [
    {"role" : "user", "content" : "Continue the sequence: 1, 1, 2, 3, 5, 8,"}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 1000, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

The sequence you're referring to is the Fibonacci sequence, in which each number is the sum of the two preceding numbers. The next numbers in the sequence would be:

13, 21, 34, 55, 89, 144,...

So, the continued sequence is: 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144.<|eot_id|>


## Save LoRA Adapters Locally

Save the trained LoRA adapter weights separately. This allows loading the adapters on top of the base model later without the full quantized weights.

In [21]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/tokenizer.json')

## Save Full Quantized Model with TorchAO

Save the complete FP8 quantized model using Unsloth's `save_pretrained_torchao`. 

**Note on File Format**: The model is saved in PyTorch `.bin` format instead of safetensors because **safetensors doesn't support FP8 tensor types**. Safetensors only supports standard dtypes (float16, bfloat16, float32, int8, uint8), while FP8 quantized tensors from torchao use custom tensor subclasses that require PyTorch's native serialization.

In [22]:
model.save_pretrained_torchao(
    "model-torchao",
    tokenizer,
)

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...


Unsloth: Copying 4 files from cache to `model-torchao`: 100%|██████████| 4/4 [00:04<00:00,  1.05s/it]


Successfully copied all 4 files from cache to `model-torchao`
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [01:45<00:00, 26.43s/it]


Unsloth: Merge process complete. Saved to `/root/src/github.com/elizabetht/language-modeling-from-scratch/quantization/unsloth/model-torchao`


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Upload to Hugging Face Hub

Create a comprehensive model card with training details and upload the quantized model to Hugging Face Hub. The model card includes:
- Base model information
- Training configuration (LoRA rank, learning rate, batch size)
- QAT scheme used (fp8-int4)
- Usage instructions
- Model performance details

In [24]:
from dotenv import load_dotenv
from huggingface_hub import login, ModelCard, ModelCardData, create_repo, upload_folder

load_dotenv()
login(token=os.getenv("HF_TOKEN"))

# Generate model card
repo_id = "tokenlabsdotrun/Llama-3.1-8B-Unsloth-FP8_INT4-QAT"
hf_write_token = os.getenv("HF_WRITE_TOKEN")

card_data = ModelCardData(
    language="en",
    license="llama3.1",
    base_model="meta-llama/Llama-3.1-8B-Instruct",
    tags=["fp8-int4", "qat", "quantization-aware-training", "unsloth", "llama", "lora"],
    pipeline_tag="text-generation",
)

card_content = f"""
---
{card_data.to_yaml()}
---

# Llama-3.1-8B FP8 QAT Fine-tuned with Unsloth

This model was fine-tuned using **FP8-INT4 Quantization-Aware Training (QAT)** with [Unsloth](https://github.com/unslothai/unsloth).

## Model Details

- **Base Model**: meta-llama/Llama-3.1-8B-Instruct
- **Fine-tuning Method**: LoRA + FP8-INT4 QAT
- **QAT Scheme**: fp8-int4
- **LoRA Rank**: 32
- **Training Dataset**: mlabonne/FineTome-100k

## What is FP8-INT4 QAT?

FP8-INT4 Quantization-Aware Training trains the model to be robust to FP8-INT4 precision loss by simulating quantization during training. This results in:
- Minimal accuracy degradation when deployed in FP8-INT4
- Faster inference with FP8-INT4 hardware support
- Reduced memory footprint

## Usage

```python
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="{repo_id}",
    dtype=torch.bfloat16,
    max_seq_length=2048,
)
```

## Training Configuration

- Learning rate: 2e-5
- Batch size: 1 (with gradient accumulation of 4)
- Optimizer: AdamW 8-bit
- Training steps: 30

## License

This model is released under the Llama 3.1 Community License.
"""

card = ModelCard(card_content)

# Upload to HuggingFace Hub
try:
    # Create the repo (set private=True if you want it private)
    create_repo(repo_id, exist_ok=True, private=False, token=hf_write_token)
    print(f"✓ Repository created: {repo_id}")
    
    # Push model card
    card.push_to_hub(repo_id, token=hf_write_token)
    print(f"✓ Model card uploaded")
    
    # Upload all files from local save
    model_path = os.path.join(os.getcwd(), "model-torchao-torchao")
    print(f"Uploading from: {model_path}")
    upload_folder(
        folder_path=model_path,
        repo_id=repo_id,
        repo_type="model",
        commit_message="Upload Llama-3.1-8B fine-tuned with Unsloth FP8 QAT",
        token=hf_write_token
    )
    print(f"✓ Uploaded to https://huggingface.co/{repo_id}")
    
except Exception as e:
    print(f"❌ Error: {e}")

No files have been modified since last commit. Skipping to prevent empty commit.
[huggingface_hub.hf_api|WARNING]No files have been modified since last commit. Skipping to prevent empty commit.


✓ Repository created: tokenlabsdotrun/Llama-3.1-8B-Unsloth-FP8_INT4-QAT
✓ Model card uploaded
Uploading from: /root/src/github.com/elizabetht/language-modeling-from-scratch/quantization/unsloth/model-torchao-torchao


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

✓ Uploaded to https://huggingface.co/tokenlabsdotrun/Llama-3.1-8B-Unsloth-FP8_INT4-QAT
